## This notebook looks at how adding some flux contamination from another source affect redshift estimation

We are going modify the fluxes of the objects but adding a small amount of flux from a different object,
while keeping the total flux of the object the same.  

This will slightly change the spectrum of the object to look a little bit more like the spectrum of the contaminating object.
We will characterize this effect as a function of the relative amount of contamination.

#### Standard imports

In [1]:
import os
import tables_io
import numpy as np
import matplotlib.pyplot as plt
import pyccl as ccl
from macss import utility_functions
from macss import plotting_functions
from macss import admixture_functions

from scipy.stats import sigmaclip
from astropy.stats import biweight_location, biweight_scale
from sklearn.neighbors import KNeighborsRegressor, RadiusNeighborsRegressor

#### Change this to have the correct location

In [2]:
HOME = os.environ['HOME']
pz_dir = f'{HOME}/macss'

#### Read test and training data, and a collection of objects to serve as library for the contamination

In [ ]:
train = tables_io.read(f"{pz_dir}/data/dp1_matched_v4_train.hdf5")
test = tables_io.read(f"{pz_dir}/data/dp1_matched_v4_test.hdf5")
d = tables_io.read(f"{pz_dir}/data/dp1_v29.0.0_gold_ECDFS.hdf5")

#### Extract test and training data

In [ ]:
train_features = utility_functions.get_band_values(train, '{band}_gaap1p0Mag', 'ugrizy')
train_targets = train['redshift']
test_features = utility_functions.get_band_values(test, '{band}_gaap1p0Mag', 'ugrizy')
test_targets = test['redshift']

#### Set up a ML algorithm and a helper function to run it and make a nice plot

In [ ]:
knr = KNeighborsRegressor()

In [ ]:
def run_it(reg):
    preds = utility_functions.run_regression(reg, train_features, train_targets, test_features)
    _ = plotting_functions.plot_true_predict_fancy(test_targets, np.nan_to_num(preds, -0.4))

#### Get a regression model

In [ ]:
run_it(knr)

#### Make a library of data to draw the contamination from

In [ ]:
library = utility_functions.get_band_values(d, '{band}_gaap1p0Mag', 'ugrizy')

#### We are going to evaluate the effect for many different levels of flux contamination

We set up that grid here

In [ ]:
admix_grid = np.logspace(-4, 0, 17)

#### Set up a function to iterate over the different levels of contamination and see how the affect the redshift

In [ ]:
def doit(reg, nclip=1):
    the_dict = {}
    inputs = np.hstack([train_features])
    est_orig = reg.predict(inputs)
    means = []
    stds = []
    biweight_means = []
    biweight_stds = []
    clipped_fracs = []

    outlier_fracs = []
    for admix in admix_grid:
        mixed_mags = admixture_functions.make_admixture(train_features, library, admixture=admix)
        inputs = np.hstack([mixed_mags])
        ad_vals = reg.predict(inputs)
        deltas = (ad_vals - est_orig)/(1 + est_orig)
        subset_clip, _, _ = sigmaclip(deltas, low=3, high=3)
        for _j in range(nclip):
            subset_clip, _, _ = sigmaclip(subset_clip, low=3, high=3)

        clipped_fracs.append((len(deltas) - len(subset_clip))/(len(deltas)))
        the_dict[admix] = deltas
        outliers = (np.fabs(deltas) > 0.15).sum() / float(deltas.size)
        outlier_fracs.append(outliers)
        means.append(deltas.mean())
        stds.append(deltas.std())
        biweight_means.append(biweight_location(subset_clip))
        biweight_stds.append(biweight_scale(subset_clip))

    _ = plt.plot(admix_grid, means, label=r"mean $\delta z$", ls='-.', color='blue')
    _ = plt.plot(admix_grid, stds, label=r"RMS $\delta z$", ls='-.', color='green')
    _ = plt.plot(admix_grid, biweight_means, label=r"biweight mean $\delta z$", ls='-', color='blue')
    _ = plt.plot(admix_grid, biweight_stds, label=r"biweight RMS $\delta z$", ls='-', color='green')    
    _ = plt.plot(admix_grid, outlier_fracs, label=r"f $\frac{\delta z}{1+z} > 0.15$", ls='-', color='cyan')
#    _ = plt.plot(admix_grid, clipped_fracs, label=r"$f_{\rm outlier}$", ls='-.', color='cyan')
    
    _ = plt.xscale('log')
    _ = plt.legend()
    _ = plt.xlabel("Flux Admixture Fraction")
    _ = plt.ylabel(r"$\delta z$")

In [ ]:
doit(knr)